In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

# Read in Data files
country_land = pd.read_csv('data/country_land.csv')
land_use_df = pd.read_csv('data/global-land-use-since-10000bc.csv')




In [2]:
# Pivot the dataframe to turn the 'Entity' values into columns
pivoted_df = land_use_df.pivot(index='Year', columns='Entity', values='area_aggregated_categories')

# Reset the index
pivoted_df.reset_index(inplace=True)



In [3]:
# Filter out the rows where 'Year' is before -2000
filtered_df = pivoted_df[pivoted_df['Year'] >= -2000].copy()



In [4]:

filtered_df.loc[:, 'Entity'] = 'World'
filtered_df.loc[:, 'Code'] = 'OWID_WRL'



In [5]:
# Reorder the columns
filtered_df = filtered_df[['Entity', 'Code', 'Year', 'Cropland', 'Pasture', 'Permanent ice', 'Semi-natural land', 'Urban', 'Villages', 'Wild barren land', 'Wild woodlands']]


In [6]:
# Function to use global context
pysqldf = lambda q: sqldf(q, globals())


In [7]:
query = """
    SELECT *
    FROM merged_df
    LEFT JOIN filtered_df
    ON merged_df.Entity = filtered_df.Entity 
    AND merged_df.Code = filtered_df.Code
    AND merged_df.Year = filtered_df.Year
    UNION ALL
    SELECT *
    FROM filtered_df
    LEFT JOIN merged_df
    ON merged_df.Entity = filtered_df.Entity 
    AND merged_df.Code = filtered_df.Code
    AND merged_df.Year = filtered_df.Year
    WHERE merged_df.Entity IS NULL
"""

In [8]:
# Execute the query
final_df_all_years = pysqldf(query)

ObjectNotExecutableError: Not an executable object: '\n    SELECT *\n    FROM merged_df\n    LEFT JOIN filtered_df\n    ON merged_df.Entity = filtered_df.Entity \n    AND merged_df.Code = filtered_df.Code\n    AND merged_df.Year = filtered_df.Year\n    UNION ALL\n    SELECT *\n    FROM filtered_df\n    LEFT JOIN merged_df\n    ON merged_df.Entity = filtered_df.Entity \n    AND merged_df.Code = filtered_df.Code\n    AND merged_df.Year = filtered_df.Year\n    WHERE merged_df.Entity IS NULL\n'

In [10]:
clean_combined.to_csv('data/cleancombined.csv')